In [1]:
from html import escape

In [13]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a) ))

def html_real(a):
    return '{0:.2f}'.format(round(a,2))

def html_str(s):
    return html_escape(s).replace('\n','<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(htmlize(k),htmlize(v) )
            for k,v in d.items()
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [15]:
print(html_escape("78a"))

print(html_int(56))

print(html_real(5.6))

print(html_list([1,2,3,4]))

print(html_dict({1:1,2:3}))

78a
56(<i>0x38</i>)
5.60
<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
<li>4(<i>0x4</i>)</li>
</ul>
<ul>
<li>1(<i>0x1</i>)=1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)=3(<i>0x3</i>)</li>
</ul>


In [10]:
from decimal import Decimal
def htmlize(arg):
    if isinstance(arg,int):
        return html_int(arg)
    elif isinstance(arg,float) or isinstance(arg,Decimal):
        return html_real(arg)
    elif isinstance(arg,str):
        return html_str(arg)
    elif isinstance(arg,list) or isinstance(arg,tuple):
        return html_list(arg)
    elif isinstance(arg,dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [16]:
htmlize("""
sdlflds 
is 
2nd line""")

'<br/>\nsdlflds <br/>\nis <br/>\n2nd line'

In [17]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int : html_int,
        str : html_str,
        list : html_list,
        tuple : hmtl_list
    }
    
    fn = registry.get(type(arg),registry[object])
    
    return fn(arg)

# limitation - we have to change the registry dicitonary every time we need to add new item.
# this is a hardcoded dictionary
# so we need a way to modify the dictionary from outside the htmlize funciton - idea of Closure !!!


In [18]:
def single_dispatch(fn):
    registry = {}
    
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [19]:
def single_dispatch(fn):
    registry = {}
    
    registry[object] = fn
    registry[int] = lambda a:  '{0}(<i>{1}</i>)'.format(a,str(a))
    registry[str] = lambda s:   html_escape(s).replace('\n','<br/>\n')
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [20]:
def single_dispatch(fn):
    
    registry = {}
    registry[object] = fn
    
    # returning the result of htmlizing
    def decorated(arg):
        return registry.get(type(arg),registry[object])
    
    # register new type to the dictionary -
    # all its doing is the fact that it has access to registry dictionary it is using that to add new fn for the type_
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    return decorated

In [21]:
# in above we dont have a handle to register function . So how to get a handle to this
def single_dispatch(fn):
    
    registry = {}
    registry[object] = fn
    
    # returning the result of htmlizing
    def decorated(arg):
        return registry.get(type(arg),registry[object])
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    # now we have a handle to register function
    decorated.register = register
    
    return decorated

In [22]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [24]:
htmlize("string")

<function __main__.htmlize>

In [25]:
htmlize.register

<function __main__.single_dispatch.<locals>.register>

In [27]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,str(a))

In [29]:
#we can stack 
@htmlize.register(list)
@htmlize.register(tuple)
def html_sequence(l):
    items = ('<li>{0}</li>'.format(htmlize(item))
            for item in l
            )
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'


In [30]:
# in above we dont have a handle to register function . So how to get a handle to this
def single_dispatch(fn):
    
    registry = {}
    registry[object] = fn
    
    # returning the result of htmlizing
    def decorated(arg):
        return registry.get(type(arg),registry[object])
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    decorated.register = register
    # give direct handle to registry - but dont give direct access to dictionary - can use during debug
    decorated.registry = registry
    return decorated

In [31]:
from functools import singledispatch

In [32]:
from numbers import Integral
from collections.abc import Sequence

In [33]:
@single_dispatch
def htmlize(a):
    return escape(str(a))

In [35]:
print(htmlize.registry)
htmlize.register

{<class 'object'>: <function htmlize at 0x00000267BAE2FAE8>}


<function __main__.single_dispatch.<locals>.register>

In [37]:
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a) ))

In [47]:
htmlize(5)

<function __main__.htmlize>